M2780.002400 Machine Listening (Fall 2020)

Instructor: Kyogu Lee (kglee@snu.ac.kr)

# Assignment 8: Advanced Neural Networks

(40 points)

Due Date : This assignment is due by 12:59PM, November 24 (Tuesday)


---

Assignment

This assignment is composed of only one part: Lab assignment, you will need to write Notebook scripts and/or functions as required and submit them electronically (via **ETL**) by the end of the due date. Before the submission, please make sure that the file name is in form of **아무개_hw8.ipynb**.

In [1]:
import librosa
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd
import numpy as np
import glob
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, pooling
from keras.optimizers import Adam
import keras

# Lab (40pts)


---

1.   Speech MNIST using CNN (20 pts)

  a) Record your speech saying zero, one, two, ..., nine. (You don't need to submit these recordings) Refer to the lab8 materials to create and learn a neural network that classifies spoken digit dataset. Use the trained model to classify your recordings and report the results. 

  b) Improve the network performance using the techniques introduced in lab8. You can use more or different types of layers, regularization. Data augmentation methods would be also helpful. Compare the results with a).

In [42]:
# a)

model = keras.models.load_model("mnist_trained")

Y_my_recording = np.eye(10)  # right answer of my recording

# preprocess recorded data and stack to make X_my_recording
X = []
for i in range(10):
    wave = np.load('./my_recording/my_{}.npy'.format(i))
    mel = librosa.feature.melspectrogram(wave, sr=8000, n_mels=80)
    X.append(mel)
data_length = []
for item in X:
    data_length.append(np.shape(item)[1])
def pad(mel, max_length):
    if np.shape(mel)[1] > max_length:
        return mel[:, :max_length]
    else:
        return np.concatenate((mel, np.zeros((80, max_length-np.shape(mel)[1]))), axis=1)
max_length = 36
X_pad = []
for item in X:
    X_pad.append(pad(item, max_length))
X_pad = np.asarray(X_pad)
X_pad = np.expand_dims(X_pad, axis=3)
X_my_recording = X_pad


score = model.evaluate(X_my_recording, Y_my_recording, verbose=0)
print("Accuracy with my recording:", score[1])

Accuracy with my recording: 0.699999988079071


In [39]:
# b) 

# prepare train data with data augmentation

import glob
import librosa
from sklearn.model_selection import train_test_split

speech_mnist_data = []
speech_mnist_target = []

speech_mnist_aug_data = []
speech_mnist_aug_target = []

speech_mnist = glob.glob('./lab8/speech_mnist/*.npy')
speech_mnist_spec_aug = glob.glob('./lab8/speech_mnist/*.npy')
for item in speech_mnist:
  target, speaker, index = item.split('/')[-1].split('.npy')[0].split('_')
  wavs = np.load(item)
  mel = librosa.feature.melspectrogram(wavs, sr=8000, n_mels=80)
  speech_mnist_data.append(mel)
  speech_mnist_target.append(np.eye(10)[(int)(target)])

for item in speech_mnist_spec_aug:
    target, speaker, index = item.split('/')[-1].split('.npy')[0].split('_')
    wavs = np.load(item)
    mel = librosa.feature.melspectrogram(wavs, sr=8000, n_mels = 80)
    f_frame = np.random.randint(0, 20)
    t_frame = np.random.randint(0, 3)
    f_loc = np.random.randint(0, mel.shape[0] - f_frame)
    t_loc = np.random.randint(0, mel.shape[1] - t_frame)
    for i in range(0, f_frame):
        mel[f_loc + i, :] = 0
    for i in range(0, t_frame):
        mel[:, t_loc + t_frame] = 0
    speech_mnist_aug_data.append(mel)
    speech_mnist_aug_target.append(np.eye(10)[(int)(target)])

data_length = []
for item in speech_mnist_data:
  data_length.append(np.shape(item)[1])

def pad(mel, max_length):
  if np.shape(mel)[1] > max_length:
    return mel[:,:max_length]
  else:
    return np.concatenate((mel, np.zeros((80, max_length-np.shape(mel)[1]))), axis=1)

max_length = np.max(data_length)
speech_mnist_data_pad = []
speech_mnist_aug_data_pad = []
for item in speech_mnist_data:
  speech_mnist_data_pad.append(pad(item, max_length))
for item in speech_mnist_aug_data:
    speech_mnist_aug_data_pad.append(pad(item, max_length))

speech_mnist_data_pad = np.asarray(speech_mnist_data_pad)
speech_mnist_data_pad = np.expand_dims(speech_mnist_data_pad, axis=3)
speech_mnist_aug_data_pad = np.asarray(speech_mnist_aug_data_pad)
speech_mnist_aug_data_pad = np.expand_dims(speech_mnist_aug_data_pad, axis=3)
speech_mnist_data_all_pad = np.concatenate((speech_mnist_data_pad, speech_mnist_aug_data_pad), axis=0)
speech_mnist_target_all = np.concatenate((speech_mnist_target, speech_mnist_aug_target), axis=0)
speech_mnist_target = np.asarray(speech_mnist_target)

X_train, _, Y_train, _ = train_test_split(speech_mnist_data_all_pad, speech_mnist_target_all, test_size=0.2, random_state=0)

In [40]:
# create new model(decreased the dropout rate)

new_model = Sequential()
new_model.add(Conv2D(32,(3,3),activation='relu', input_shape=(80,36,1)))
new_model.add(Conv2D(32,(3,3),activation='relu'))
new_model.add(pooling.MaxPooling2D(pool_size=(2,2)))
new_model.add(Dropout(0.2))
new_model.add(Conv2D(32,(3,3),activation='relu'))
new_model.add(Conv2D(32,(3,3),activation='relu'))
new_model.add(pooling.MaxPooling2D(pool_size=(2,2)))
new_model.add(Dropout(0.2))
new_model.add(Conv2D(32,(3,3),activation='relu'))
new_model.add(Conv2D(32,(3,3),activation='relu'))
new_model.add(pooling.MaxPooling2D(pool_size=(2,2)))
new_model.add(Dropout(0.2))


new_model.add(Flatten())
new_model.add(Dense(128, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(10, activation='softmax'))

opt = Adam(lr=0.001)
new_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])



In [41]:
print(X_train.shape)
print(Y_train.shape)
hist = new_model.fit(X_train, Y_train, batch_size=64, epochs=50, validation_split = 0.1, verbose=1)

(3196, 80, 36, 1)
(3196, 10)
Epoch 1/50
45/45 [==============================] - 4s 96ms/step - loss: 2.2481 - accuracy: 0.1380 - val_loss: 2.0885 - val_accuracy: 0.2500
Epoch 2/50
45/45 [==============================] - 4s 96ms/step - loss: 2.0933 - accuracy: 0.2330 - val_loss: 1.8104 - val_accuracy: 0.4094
Epoch 3/50
45/45 [==============================] - 4s 97ms/step - loss: 1.9932 - accuracy: 0.2987 - val_loss: 1.7061 - val_accuracy: 0.4344
Epoch 4/50
45/45 [==============================] - 4s 96ms/step - loss: 1.8265 - accuracy: 0.3428 - val_loss: 1.4124 - val_accuracy: 0.4844
Epoch 5/50
45/45 [==============================] - 4s 97ms/step - loss: 1.5602 - accuracy: 0.4350 - val_loss: 1.1692 - val_accuracy: 0.6031
Epoch 6/50
45/45 [==============================] - 4s 96ms/step - loss: 1.3510 - accuracy: 0.5170 - val_loss: 1.0398 - val_accuracy: 0.6094
Epoch 7/50
45/45 [==============================] - 4s 97ms/step - loss: 1.2948 - accuracy: 0.5348 - val_loss: 0.9404 - val_a

In [43]:
score = new_model.evaluate(X_my_recording, Y_my_recording, verbose=0)
print("Accuracy with my recording:", score[1])

Accuracy with my recording: 1.0



---


2.   Music Composer using LSTM (20 pts)


Refer to the lab8 materials to create and learn a neural network that composes music autoregressively.  Change the input feature and model structure to use 50 notes to generate the next note. Use the trained model to generate 500 notes of music starting from the first 50 notes of DOS.mid on the dataset. Save and submit the generated music as midi file. Your submission should include the trained model weights as hdf5 file format.

In [3]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


In [ ]:
location = './lab8/'

def get_notes():
    notes = []

    for file in glob.glob(location + "midi_songs/*.mid")[:5]:
        midi = converter.parse(file)
        notes_to_parse = None
        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open(location + 'data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes


notes = get_notes()
n_vocab = len(set(notes))


In [4]:
location = './lab8/'
with open('./notes_colab', 'rb') as filepath:
    notes = pickle.load(filepath)

pitchnames = sorted(set(item for item in notes))
n_vocab = len(set(notes))


In [4]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 50

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # print(pitchnames)
     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    # print(note_to_int)
    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [81]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [82]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = location + "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=100, batch_size=128, callbacks=callbacks_list)


In [83]:
network_input, network_output = prepare_sequences(notes, n_vocab)
print(network_input.shape)
model = create_network(network_input, n_vocab)
train(model, network_input, network_output)

(57127, 50, 1)
Epoch 1/100
447/447 [==============================] - 463s 1s/step - loss: 5.1635
Epoch 2/100
447/447 [==============================] - 462s 1s/step - loss: 4.6753
Epoch 3/100
447/447 [==============================] - 461s 1s/step - loss: 4.6206
Epoch 4/100
447/447 [==============================] - 462s 1s/step - loss: 4.6377
Epoch 5/100
447/447 [==============================] - 462s 1s/step - loss: 4.6275
Epoch 6/100
447/447 [==============================] - 462s 1s/step - loss: 4.5754
Epoch 7/100
447/447 [==============================] - 461s 1s/step - loss: 4.5313
Epoch 8/100
447/447 [==============================] - 462s 1s/step - loss: 4.5099
Epoch 9/100
447/447 [==============================] - 461s 1s/step - loss: 4.4723
Epoch 10/100
447/447 [==============================] - 462s 1s/step - loss: 4.4297
Epoch 11/100
447/447 [==============================] - 461s 1s/step - loss: 4.3859
Epoch 12/100
447/447 [==============================] - 462s 1s/step -

In [5]:
with open('./DOS_notes', 'rb') as filepath:
    DOS_notes = pickle.load(filepath)


In [11]:
def prepare_DOS_sequences(DOS_notes, pitchnames, n_vocab):
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    sequence_length = 50
    network_input = []
    sequence_in = DOS_notes[0:sequence_length]
    # print('sequence_in')
    # print(sequence_in)
    network_input.append([note_to_int[char] for char in sequence_in])
    # print(network_input)
    n_patterns = len(network_input)
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    normalized_input = normalized_input / float(n_vocab)
    return (network_input, normalized_input)

def load_network(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    model.load_weights(location + 'weights-improvement-100-0.4912-bigger.hdf5')
    return model

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    start = 0

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    pattern = network_input[start]
    prediction_output = []
    
    # generate 500 notes
    for note_index in range(500):
        if note_index == 0:
            for i in range(50):
                prediction_output.append(int_to_note[network_input[0][i]])
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output
    
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp=location + 'test_output.mid')

In [12]:
network_input, normalized_input = prepare_DOS_sequences(DOS_notes, pitchnames, n_vocab)
model = load_network(normalized_input, n_vocab)
prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
concat_with_input = []

create_midi(prediction_output)
